In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Bank Customer Churn Prediction ML Model
# Complete implementation with 8 algorithms and ensemble approach

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

# Load and explore data
print("Loading Bank Churn Dataset...")
data = pd.read_csv('Churn_Modelling.csv')

print(f"Dataset shape: {data.shape}")
print(f"Target distribution:\n{data['Exited'].value_counts()}")
print(f"Churn rate: {data['Exited'].mean()*100:.2f}%")

# Data preprocessing and feature engineering
def preprocess_data(df):
    """
    Comprehensive data preprocessing and feature engineering
    """
    # Create copy
    processed_df = df.copy()
    
    # Drop irrelevant columns
    processed_df = processed_df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
    
    # Feature Engineering - Create 13 engineered features
    
    # 1. Age groups
    processed_df['AgeGroup'] = pd.cut(processed_df['Age'], 
                                     bins=[0, 30, 40, 50, 60, 100], 
                                     labels=[0, 1, 2, 3, 4]).astype(int)
    
    # 2. Balance categories
    processed_df['BalanceCategory'] = pd.cut(processed_df['Balance'], 
                                            bins=[0, 1, 50000, 100000, 200000, float('inf')], 
                                            labels=[0, 1, 2, 3, 4]).astype(int)
    
    # 3. Credit score categories
    processed_df['CreditScoreCategory'] = pd.cut(processed_df['CreditScore'], 
                                                bins=[0, 600, 700, 800, 900], 
                                                labels=[0, 1, 2, 3]).astype(int)
    
    # 4. Salary categories
    processed_df['SalaryCategory'] = pd.cut(processed_df['EstimatedSalary'], 
                                           bins=[0, 50000, 100000, 150000, float('inf')], 
                                           labels=[0, 1, 2, 3]).astype(int)
    
    # 5. Balance per product ratio
    processed_df['BalancePerProduct'] = processed_df['Balance'] / (processed_df['NumOfProducts'] + 0.001)
    
    # 6. Tenure to age ratio
    processed_df['TenureAgeRatio'] = processed_df['Tenure'] / processed_df['Age']
    
    # 7. Is senior citizen (Age > 60)
    processed_df['IsSenior'] = (processed_df['Age'] > 60).astype(int)
    
    # 8. Has high balance (Balance > 100k)
    processed_df['HasHighBalance'] = (processed_df['Balance'] > 100000).astype(int)
    
    # 9. Credit score to age ratio
    processed_df['CreditAgeRatio'] = processed_df['CreditScore'] / processed_df['Age']
    
    # 10. Product engagement score
    processed_df['EngagementScore'] = (processed_df['HasCrCard'] + 
                                      processed_df['IsActiveMember'] + 
                                      processed_df['NumOfProducts']) / 3
    
    # 11. Financial stability score
    processed_df['FinancialStability'] = (processed_df['Balance'] / 100000 + 
                                         processed_df['EstimatedSalary'] / 100000) / 2
    
    # 12. Tenure category
    processed_df['TenureCategory'] = pd.cut(processed_df['Tenure'], 
                                           bins=[0, 2, 5, 8, 10], 
                                           labels=[0, 1, 2, 3]).astype(int)
    
    # 13. High value customer indicator
    processed_df['IsHighValue'] = ((processed_df['Balance'] > 100000) | 
                                  (processed_df['EstimatedSalary'] > 150000)).astype(int)
    
    # Label encode categorical variables
    le_geography = LabelEncoder()
    le_gender = LabelEncoder()
    
    processed_df['Geography'] = le_geography.fit_transform(processed_df['Geography'])
    processed_df['Gender'] = le_gender.fit_transform(processed_df['Gender'])
    
    return processed_df, le_geography, le_gender

# Preprocess data
df_processed, le_geo, le_gen = preprocess_data(data)

print(f"\nAfter feature engineering: {df_processed.shape}")
print(f"Total features created: {df_processed.shape[1] - 1}")  # -1 for target variable

# Prepare features and target
X = df_processed.drop('Exited', axis=1)
y = df_processed['Exited']

print(f"\nFeature columns ({len(X.columns)}):")
print(X.columns.tolist())

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nDataset split:")
print(f"Training: {X_train.shape[0]} samples ({y_train.mean()*100:.2f}% churn)")
print(f"Testing: {X_test.shape[0]} samples ({y_test.mean()*100:.2f}% churn)")

# Feature scaling for algorithms that need it
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model evaluation function
def evaluate_model(y_true, y_pred, y_pred_proba, model_name):
    """Comprehensive model evaluation"""
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred_proba)
    
    print(f"Accuracy: {accuracy*100:.2f}%")
    print(f"Precision: {precision*100:.2f}%")
    print(f"Recall: {recall*100:.2f}%")
    print(f"F1-Score: {f1*100:.2f}%")
    print(f"ROC-AUC: {auc:.4f}")
    
    return accuracy, precision, recall, f1, auc

# Store results
results = {}

print("\n" + "="*80)
print("IMPLEMENTING 8 MACHINE LEARNING ALGORITHMS")
print("="*80)

# 1. LOGISTIC REGRESSION
print("\n" + "="*50)
print("1. LOGISTIC REGRESSION (Baseline)")
print("="*50)

lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train_scaled, y_train)
lr_pred = lr_model.predict(X_test_scaled)
lr_pred_proba = lr_model.predict_proba(X_test_scaled)[:, 1]

lr_metrics = evaluate_model(y_test, lr_pred, lr_pred_proba, 'Logistic Regression')
results['Logistic Regression'] = lr_metrics

# 2. RANDOM FOREST
print("\n" + "="*50)
print("2. RANDOM FOREST")
print("="*50)

rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_pred_proba = rf_model.predict_proba(X_test)[:, 1]

rf_metrics = evaluate_model(y_test, rf_pred, rf_pred_proba, 'Random Forest')
results['Random Forest'] = rf_metrics

# 3. XGBOOST
print("\n" + "="*50)
print("3. XGBOOST")
print("="*50)

xgb_model = XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    eval_metric='logloss'
)

xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)
xgb_pred_proba = xgb_model.predict_proba(X_test)[:, 1]

xgb_metrics = evaluate_model(y_test, xgb_pred, xgb_pred_proba, 'XGBoost')
results['XGBoost'] = xgb_metrics

# 4. LIGHTGBM
print("\n" + "="*50)
print("4. LIGHTGBM")
print("="*50)

lgb_model = LGBMClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

lgb_model.fit(X_train, y_train)
lgb_pred = lgb_model.predict(X_test)
lgb_pred_proba = lgb_model.predict_proba(X_test)[:, 1]

lgb_metrics = evaluate_model(y_test, lgb_pred, lgb_pred_proba, 'LightGBM')
results['LightGBM'] = lgb_metrics

# 5. CATBOOST
print("\n" + "="*50)
print("5. CATBOOST")
print("="*50)

catboost_model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    depth=6,
    random_seed=42,
    verbose=False
)

catboost_model.fit(X_train, y_train)
catboost_pred = catboost_model.predict(X_test)
catboost_pred_proba = catboost_model.predict_proba(X_test)[:, 1]

catboost_metrics = evaluate_model(y_test, catboost_pred, catboost_pred_proba, 'CatBoost')
results['CatBoost'] = catboost_metrics

# 6. SUPPORT VECTOR MACHINE
print("\n" + "="*50)
print("6. SUPPORT VECTOR MACHINE (SVM)")
print("="*50)

svm_model = SVC(
    kernel='rbf',
    C=1.0,
    gamma='scale',
    probability=True,
    random_state=42
)

svm_model.fit(X_train_scaled, y_train)
svm_pred = svm_model.predict(X_test_scaled)
svm_pred_proba = svm_model.predict_proba(X_test_scaled)[:, 1]

svm_metrics = evaluate_model(y_test, svm_pred, svm_pred_proba, 'SVM')
results['SVM'] = svm_metrics

# 7. NEURAL NETWORK
print("\n" + "="*50)
print("7. NEURAL NETWORK (MLPClassifier)")
print("="*50)

nn_model = MLPClassifier(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    solver='adam',
    alpha=0.01,
    max_iter=1000,
    random_state=42
)

nn_model.fit(X_train_scaled, y_train)
nn_pred = nn_model.predict(X_test_scaled)
nn_pred_proba = nn_model.predict_proba(X_test_scaled)[:, 1]

nn_metrics = evaluate_model(y_test, nn_pred, nn_pred_proba, 'Neural Network')
results['Neural Network'] = nn_metrics

# 8. ENSEMBLE MODEL (VOTING CLASSIFIER)
print("\n" + "="*50)
print("8. ENSEMBLE MODEL (Voting Classifier)")
print("="*50)

# Create pipelines for models that need scaling
lr_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression(random_state=42, max_iter=1000))
])

svm_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(kernel='rbf', C=1.0, gamma='scale', probability=True, random_state=42))
])

nn_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('nn', MLPClassifier(hidden_layer_sizes=(100, 50), activation='relu', 
                        solver='adam', alpha=0.01, max_iter=1000, random_state=42))
])

# Create ensemble with 6 best performing models
ensemble_model = VotingClassifier([
    ('rf', rf_model),
    ('xgb', xgb_model),
    ('lgb', lgb_model),
    ('catboost', catboost_model),
    ('svm', svm_pipeline),
    ('nn', nn_pipeline)
], voting='soft')

ensemble_model.fit(X_train, y_train)
ensemble_pred = ensemble_model.predict(X_test)
ensemble_pred_proba = ensemble_model.predict_proba(X_test)[:, 1]

ensemble_metrics = evaluate_model(y_test, ensemble_pred, ensemble_pred_proba, 'Ensemble')
results['Ensemble'] = ensemble_metrics

# RESULTS COMPARISON
print("\n" + "="*80)
print("COMPREHENSIVE RESULTS COMPARISON")
print("="*80)

# Create results dataframe
results_df = pd.DataFrame.from_dict(results, orient='index', 
                                   columns=['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC'])

# Sort by F1-Score
results_df = results_df.sort_values('F1-Score', ascending=False)

print(results_df.round(4))

print(f"\n🏆 BEST MODEL (F1-Score): {results_df.index[0]}")
best_model = results_df.iloc[0]
print(f"   📊 Accuracy: {best_model['Accuracy']*100:.2f}%")
print(f"   🎯 Precision: {best_model['Precision']*100:.2f}%")
print(f"   🔍 Recall: {best_model['Recall']*100:.2f}%")
print(f"   ⚖️  F1-Score: {best_model['F1-Score']*100:.2f}%")
print(f"   📈 ROC-AUC: {best_model['ROC-AUC']:.4f}")

# Calculate False Positive Rate for best models
print(f"\n📉 FALSE POSITIVE RATE ANALYSIS:")
for model_name in results_df.index[:3]:  # Top 3 models
    precision = results_df.loc[model_name, 'Precision']
    # FPR approximation: (1 - precision) * positive_predictions / total_negative
    fpr_approx = (1 - precision) * 100
    print(f"   {model_name}: ~{fpr_approx:.1f}%")

# CROSS-VALIDATION FOR ENSEMBLE
print("\n" + "="*50)
print("5-FOLD CROSS-VALIDATION (ENSEMBLE)")
print("="*50)

cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Cross-validation scores
cv_accuracy = cross_val_score(ensemble_model, X, y, cv=cv_strategy, scoring='accuracy', n_jobs=-1)
cv_precision = cross_val_score(ensemble_model, X, y, cv=cv_strategy, scoring='precision', n_jobs=-1)
cv_recall = cross_val_score(ensemble_model, X, y, cv=cv_strategy, scoring='recall', n_jobs=-1)
cv_f1 = cross_val_score(ensemble_model, X, y, cv=cv_strategy, scoring='f1', n_jobs=-1)
cv_auc = cross_val_score(ensemble_model, X, y, cv=cv_strategy, scoring='roc_auc', n_jobs=-1)

print(f"Cross-Validation Results:")
print(f"Accuracy:  {cv_accuracy.mean():.4f} (+/- {cv_accuracy.std() * 2:.4f})")
print(f"Precision: {cv_precision.mean():.4f} (+/- {cv_precision.std() * 2:.4f})")
print(f"Recall:    {cv_recall.mean():.4f} (+/- {cv_recall.std() * 2:.4f})")
print(f"F1-Score:  {cv_f1.mean():.4f} (+/- {cv_f1.std() * 2:.4f})")
print(f"ROC-AUC:   {cv_auc.mean():.4f} (+/- {cv_auc.std() * 2:.4f})")

# FEATURE IMPORTANCE (from Random Forest)
print("\n" + "="*50)
print("TOP 10 MOST IMPORTANT FEATURES")
print("="*50)

feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print(feature_importance.head(10).to_string(index=False))

# FINAL MODEL PERFORMANCE SUMMARY
print("\n" + "="*80)
print("🎯 FINAL MODEL PERFORMANCE SUMMARY")
print("="*80)

print(f"✅ Dataset Size: {len(data):,} customers")
print(f"✅ Features Engineered: 13 custom features")
print(f"✅ Algorithms Implemented: 8 (LR, RF, XGB, LGB, CatBoost, SVM, NN, Ensemble)")
print(f"✅ Best Model: {results_df.index[0]}")
print(f"✅ Best F1-Score: {results_df.iloc[0]['F1-Score']*100:.1f}%")
print(f"✅ Cross-Validation: 5-fold stratified")
print(f"✅ Model Improvement: {((results_df.iloc[0]['F1-Score'] - results['Logistic Regression'][3])/results['Logistic Regression'][3]*100):.1f}% over baseline")

print(f"\n🚀 PRODUCTION READY: {results_df.index[0]} Model")
print(f"   Ready for deployment with {results_df.iloc[0]['F1-Score']*100:.1f}% F1-Score!")

print("\n" + "="*80)
print("MODEL TRAINING COMPLETED SUCCESSFULLY! 🎉")
print("="*80)

Loading Bank Churn Dataset...


FileNotFoundError: [Errno 2] No such file or directory: 'Churn_Modelling.csv'